In [1]:
%cd /content/drive/MyDrive/
# raw_data is imported from global config
root_dir='/content/drive/MyDrive/'  # ChangeThis
result_dir=root_dir+'grasp/TSception/skorchL1Norm/'

/content/drive/MyDrive


In [2]:
import sys, importlib
#importlib.reload(sys.modules['grasp.config'])
from grasp.config import *
# orveride the data_dir in config file
#data_dir='/content/drive/MyDrive/data/' # googleDrive

In [3]:
data_dir

'/content/drive/MyDrive/data/'

In [4]:
! pip install mne==0.19.2
! pip install torch
! pip install skorch

In [5]:
import torch
from skorch.helper import predefined_split
from torch import nn
import torch.nn.functional as F
from skorch import NeuralNetRegressor
from skorch.callbacks import Callback
import numpy as np
import matplotlib.pyplot as plt

from grasp.TSception.Models import TSception2
from grasp.utils import rawData2,SEEGDataset,set_random_seeds,cuda_or_cup
from grasp.config import activeChannels, root_dir, tmp_dir
from grasp.myskorch import plotPrediction, MyRegressor


In [35]:
# reload function
import sys, importlib
importlib.reload(sys.modules['grasp.TSception.Models'])
importlib.reload(sys.modules['grasp.utils'])
from grasp.TSception.Models import TSception,TSception2
from grasp.utils import rawData2,SEEGDataset

In [7]:
# import from another folder
import sys
sys.path.insert(1, '/content/drive/MyDrive/examples')
from IMV_LSTM.networks import IMVTensorLSTM

In [8]:
import inspect as i
import sys
#sys.stdout.write(i.getsource(rawData2));

In [9]:
device=cuda_or_cup()
seed = 123456789  # random seed to make results reproducible
# Set random seed to be able to reproduce results
set_random_seeds(seed=seed)


GPU computing:   True


In [12]:
%%capture
# suppress the output
sampling_rate=1000
traindata, valdata, testdata = rawData2('band','all',move2=True)  # (chns, 15000/15001, 118) (channels, time, trials)
##traindata, valdata, testdata = rawData2('raw','all',move2=True)
traindata = traindata.transpose(2, 0, 1)  # (118, 20, 15000) (trials,channels,  time)
valdata = valdata.transpose(2, 0, 1) # (8, 20, 15000)
testdata = testdata.transpose(2, 0, 1)  # (8, 20, 15000)


In [37]:
trainx, trainy = traindata[:, :-2, :], traindata[:, -2, :] #-2 is real force, -1 is target
valx, valy = valdata[:, :-2, :], valdata[:, -2, :]
testx, testy = testdata[:, :-2, :], testdata[:, -2, :]

samples=trainx.shape[0]
chnNum=trainx.shape[1]
step=50 #ms
T=1000 #ms
totalLen=trainx.shape[2] #ms
batch_size=int((totalLen-T)/step) # 280

train_ds = SEEGDataset(trainx, trainy,T, step)
val_ds = SEEGDataset(valx, valy,T, step)
test_ds = SEEGDataset(testx, testy,T, step)


In [38]:
learning_rate=0.002
num_T = 3 # (6 conv2d layers) * ( 3 kernel each layer)
num_S = 3
dropout=0.5
Lambda = 1e-6
model=TSception2(T, step, sampling_rate,chnNum, num_T, num_S,batch_size,dropout).float()

#if(enable_cuda):
#	model.cuda()
 
#checkpoint = torch.load(result_dir+'checkpoint195.pth')
#net.load_state_dict(checkpoint['state_dict'])
#optimizer.load_state_dict(checkpoint['optimizer'])

In [39]:
net = MyRegressor(
    model,
    #train_split=predefined_split(valid_set),
    iterator_train__shuffle=True,
    train_split=predefined_split(val_ds),
    max_epochs=200,
    lr=learning_rate,
    batch_size=1,
    optimizer=torch.optim.Adam,
    criterion = nn.MSELoss,
    callbacks=[('plotPrediction', plotPrediction(result_dir)),],
    device = device
)


In [40]:
net.fit(train_ds,y=None)

  epoch    train_loss    valid_loss       dur
-------  ------------  ------------  --------
      1        0.1055        0.0830  108.9078
      2        0.0978        0.0881  106.3287
      3        0.0943        0.0766  106.2852
      4        0.0945        0.0802  106.3379
      5        0.0912        0.0767  106.3983
      6        0.0898        0.0788  106.3684
      7        0.0924        0.0794  106.3839
      8        0.0927        0.0799  106.3714
      9        0.0975        0.0827  106.3711
     10        0.0945        0.0795  106.4305
     11        0.0905        0.0936  106.3012
     12        0.0950        0.0827  106.3016
     13        0.0964        0.0844  106.2519
     14        0.0967        0.0818  106.2822
     15        0.0952        0.0831  106.2900
     16        0.0969        0.0721  106.2538
     17        0.0931        0.0767  106.2702
     18        0.0932        0.0736  106.2725
     19        0.0900        0.0665  106.2793
     20        0.0883        0.062

<class 'grasp.myskorch.MyRegressor'>[initialized](
  module_=TSception2(
    (Tception1): Sequential(
      (0): Conv2d(1, 3, kernel_size=(1, 500), stride=(1, 1), padding=(0, 250))
      (1): ReLU()
      (2): AvgPool2d(kernel_size=(1, 16), stride=(1, 8), padding=0)
    )
    (Tception2): Sequential(
      (0): Conv2d(1, 3, kernel_size=(1, 250), stride=(1, 1), padding=(0, 125))
      (1): ReLU()
      (2): AvgPool2d(kernel_size=(1, 16), stride=(1, 8), padding=0)
    )
    (Tception3): Sequential(
      (0): Conv2d(1, 3, kernel_size=(1, 126), stride=(1, 1), padding=(0, 63))
      (1): ReLU()
      (2): AvgPool2d(kernel_size=(1, 16), stride=(1, 8), padding=0)
    )
    (Tception4): Sequential(
      (0): Conv2d(1, 3, kernel_size=(1, 62), stride=(1, 1), padding=(0, 31))
      (1): ReLU()
      (2): AvgPool2d(kernel_size=(1, 16), stride=(1, 8), padding=0)
    )
    (Tception5): Sequential(
      (0): Conv2d(1, 3, kernel_size=(1, 32), stride=(1, 1), padding=(0, 16))
      (1): ReLU()
      